# TAREA # 1: COMPRENDER EL PROBLEMA Y EL CASO PRÁCTICO


<table>
  <tr><td>
    <img src="https://drive.google.com/uc?id=11BquVVgQTebvVO5NZ2TGA526rulbWBv5"
         alt="Fashion MNIST sprite"  width="1000">
  </td></tr>
  <tr><td align="center">
    <b>Figura 1. Análisis del Sentimiento de los Clientes
  </td></tr>
</table>

![alt text](https://drive.google.com/uc?id=1nCRtae4P_PA29o7_ZZxT4KZ_Dwn9pVto)

![alt text](https://drive.google.com/uc?id=1KWse1Fc_seaqEfgnKMTdFRM1Xmo_ZFzb)

# TAREA # 2: IMPORTAR LIBRERÍAS Y DATA SETS

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandasgui import show

In [ ]:
# Necesitarás montar tu disco usando los siguientes comandos:
# Para obtener más información sobre el montaje, consulta en: https://stackoverflow.com/questions/46986398/import-data-into-google-colaboratory
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('amazon_alexa.tsv', sep='\t')
show(df.head())
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

Observamos que el 91% de la gente deja muy buena nota en el *rating*.

In [ ]:
df['verified_reviews']

# TAREA #3: EXPLORAR EL DATASET

### Analizamos los NAs:

In [ ]:
sns.heatmap(df.isnull(), yticklabels= False, cbar = False, cmap="Blues")
plt.show()

In [ ]:
df.hist(bins=30, figsize=(13,5), color = 'r')
plt.show()

Vemos que es un dataset bastante desbalanceado, hay muchas más valoraciones
positivas que negativas.

**Vamos a añadir una columna con la longitud de cada valoración:**

In [ ]:
df['length'] = df['verified_reviews'].apply(len)
df.head()
#show(df.head())

In [ ]:
df['length'].plot(bins = 100, kind = 'hist')
plt.show()

Vemos que hay mucha gente que escribe valoraciones cortas.

In [ ]:
df.describe()

In [ ]:
df[df['length'] == 2851]['verified_reviews'].iloc[0]

In [ ]:
df[df['length'] == 1]['verified_reviews'].iloc[0]

* Las valoraciones suelen tener una media de 132 carácteres.
* El que más ha escrito han sido 2851 palabras y el que menos 1

In [ ]:
positive = df[df['feedback'] == 1]
show(positive)
negative = df[df['feedback'] == 0]
show(negative)

In [ ]:
sns.countplot(df['feedback'], label = 'Count')
plt.show()

In [ ]:
sns.countplot(x = 'rating', data= df)
plt.show()

In [ ]:
df['rating'].hist(bins=5)
plt.show()

**Vamos a ver el campo *variation* que nos indica el tipo de producto:**

In [ ]:
plt.figure(figsize(40,15))
sns.barplot(x='variation', y = 'rating', data=df, palette='deep')
plt.show()

**1-Juntamos todas las valoraciones:**

In [ ]:
sentences =df['verified_reviews'].tolist()
len(sentences)
print(sentences)

**2- Unimos todas las frases separadas por un espacio en blanco:**

In [ ]:
sentences_as_one_string= " ".join(sentences)
sentences_as_one_string

**1-Visualizamos las palabras y su frecuencia:**

In [ ]:
from wordcloud import WordCloud

plt.figure(figsize=(20,20))
plt.imshow(WordCloud().generate(sentences_as_one_string))
plt.show()

Observamos que las palabras *Alexa* y *love* sobresalen del resto.

**Ahora vamos a hacer lo mismo pero para las palbras negativas**

In [ ]:
negative_list = negative['verified_reviews'].tolist()
negative_as_one_string = " ".join(negative_list)
plt.figure(figsize=(20,20))
plt.imshow(WordCloud().generate(negative_as_one_string))
plt.show()

# TAREA #4: LLEVAR A CABO LA LIMPIEZA DE DATOS

In [ ]:
#Eliminamos las columnas que no necesitamos
df = df.drop(['date', 'rating', 'length'], axis = 1)
df 

**Vamos a pasar el tipo de producto a dato numérico para trabjar mejor con
estos(pasamos a variables dummie):**

In [ ]:
variation_dummies = pd.get_dummies(df['variation'], drop_first=True)
variation_dummies

* Borramos la columna *variation* ya que ya no nos hace falta y agregamos la
columna dummie creada arriba:

In [ ]:
df.drop(['variation'], axis=1, inplace=True)

In [ ]:
df = pd.concat([df, variation_dummies], axis = 1)
df

# TASK #5:Eliminamos los sígnos de puntuación del texto

In [ ]:
import string
string.punctuation

**1- Frase de prueba:**

In [ ]:
test = "Hi client! Im so happy to be here with you!!!"
test_punc_remove = [char for char in Test if char not in string.punctuation]
test_punc_remove

In [ ]:
test_punc_remove_join = ''.join(test_punc_remove)
test_punc_remove_join

**2-Nuestro dataset:**

# TAREA 6: ENTENDER COMO LIAR LAS STOPWORDS

**Eliminamos las llamadas palabras de parada:** Son palabras como 'yo', 'tu',
'y' que no aportan información como los signos de puntuación.

In [ ]:
import nltk

nltk.download('stopwords')

Como depende del idioma hacemos para el inglés:

In [ ]:
from nltk.corpus import stopwords
show(stopwords.words('english'))

In [ ]:
test_punc_remove_join_clean = [word for word in test_punc_remove_join.split() if word.lower() not in stopwords.words('english')]
test_punc_remove_join_clean

# TAREA 7: ENTENDER EL PROCESO DE TOKENIZACIÓN
![alt text](https://drive.google.com/uc?id=10dlXiQMdvzLJwNqhnvylAix9_LUQCGQa)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

sample_data = ['This is the first document.', 'This is the second document.', 'And this is the third one.', 'Is this the first document?']
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(sample_data)

print(vectorizer.get_feature_names())

#version tokenizada
print(x.toarray())

# TAREA #8: LLEVAR A CABO LA LIMPIEZA DE DATOS APLICANDO TODO LO QUE HEMOS
APRENDIDO!!


### Definimos un pipeline para limpiar todos los mensajes y quedarnos con las
palabras más relevantes.

**1.1 - Forma manual**
Creamos una función que elimine los signos de puntuación y eliminamos los
stopwords.
*(los stopwords son palabras que no nos dan ninguna info relevante como
pronombres o conectores)*

In [ ]:
def message_cleaning(message):
  test_punc_remove = [char for char in message if char not in string.punctuation]
  test_punc_remove_join = ''.join(test_punc_remove)
  test_punc_remove_join_clean = [word for word in test_punc_remove_join.split() if word.lower() not in stopwords.words('english')]
  return test_punc_remove_join_clean


In [ ]:
df_clean = df['verified_reviews'].apply(message_cleaning)

#Comprarmos como nos ha quedado a como era:
print(df_clean[3])
print(df['verified_reviews'][3])

**1.2 - Aplicando una función ya prefabricada en una librería:**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer=message_cleaning)
reviews_countvectorizer = vectorizer.fit_transform(df['verified_reviews'])

print(vectorizer.get_feature_names())

**2 - Tokenizamos el vector:**

In [ ]:
print(reviews_countvectorizer.toarray())
print(reviews_countvectorizer.shape)

**3 - Eliminamos y pegamos la nueva columna tokenizada:**

In [ ]:
df.drop(['verified_reviews'], axis=1, inplace=True)

#Para añadir la nueva columna que está en array hay que pasarla a dataframe primero
reviews = pd.DataFrame(reviews_countvectorizer.toarray())

df = pd.concat([df, reviews], axis=1)

df

*4 - Separamos nuestros datos de entrada de nuestra variable a predecir:***

In [ ]:
X = df.drop(['feedback'], axis = 1)
y = df['feedback']
print(y)
print(X)

# TAREA #9: ENTENDER LA TEORÍA Y LA INTUICIÓN DETRÁS DE NAÏVE BAYES

![alt text](https://drive.google.com/uc?id=1AyMGr5pZWQ3frRD_84NAM1k765dZNm0S)

![alt text](https://drive.google.com/uc?id=1lLQR8v2Iu8-uXzC7sgNdzVAXjtAkvh2z)

![alt text](https://drive.google.com/uc?id=1KuifCAosRqw0VG0d9itxul4Yjm1LFfHX)

![alt text](https://drive.google.com/uc?id=1XOfRNtgEn6NQcDu-kats3rVMh75hrzZW)

![alt text](https://drive.google.com/uc?id=1JYeNmZFOMDWzzW82ARq851WVhi38pMmp)

![alt text](https://drive.google.com/uc?id=12kQKrWY3pvAWaLazeLSfUrgiQIisd0qX)

![alt text](https://drive.google.com/uc?id=1q3SWsZkfUtxnI6r-uqBZvK8KSnZ_R2qj)

![alt text](https://drive.google.com/uc?id=1swfSHTNQZGOwbPOSc6J775Fe8YDT7A2J)

# TAREA #10: ENTRENAR UN MODELO CLASIFICADOR DE NAÏVE BAYES

In [ ]:
print(X.shape)
print(y.shape)

### Creamos el modelo.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb_multinomial = MultinomialNB()
nb_multinomial.fit(X_train, y_train)

# TAREA #11: VALIDAR LA EFICACIA DEL MODELO ENTRENADO

![alt text](https://drive.google.com/uc?id=1ZKbO9tQH3qh0IjKwgOAJ37ghWzUVcb8O)

In [ ]:

from sklearn.metrics import classification_report, confusion_matrix

y_predict_train = nb_multinomial.predict(X_train)
y_predict_train

In [ ]:
cm = confusion_matrix(y_train, y_predict_train)
sns.heatmap(cm, annot=True)
plt.show()

In [ ]:
y_predict_test = nb_multinomial.predict(X_test)
cm = confusion_matrix(y_test, y_predict_test)
sns.heatmap(cm, annot=True)
plt.show()

In [ ]:
print(classification_report(y_test, y_predict_test))

Podemos observar que nuestro modelo capta muy bien las valoraciones positivas (tenemos un recall de 0.98) mientras que las negativas las detecta bastante mal. Ya que nuestro dataset no está muy bien balanceado.

# TAREA #12: RETO FINAL - ENTRENAR Y EVALUAR UN MODELO CLASIFICADOR BASADO EN LA
REGRESIÓN LOGÍSTICA

# UN TRABAJO EXCELENTE! PUEDES SENTIRTE ORGULLOSO DE TODO LO QUE ACABAS DE
APRENDER